# Run Inference on Multi-Region S3 Buckets

### Prerequisites
- terraform
- python
- aws cli (configured)

In [ ]:
%cd multi-region

## Add your regions to regions.md
### Note: The Bootstrap region should be first

In [2]:
%cat regions.md

# aws ec2 describe-regions | jq -r '.Regions[].RegionName'
# Bootstrap region must be first
ca-central-1
eu-north-1
us-east-1
us-west-1


In [5]:
#Replace the value of the name parameter with the name of your AMI
%python3 get_instance_amis.py --name "Deep Learning AMI Graviton GPU PyTorch 1.10.0 (Ubuntu 20.04) 20211122"

{
    "ca-central-1": {
        "region": "ca-central-1",
        "zone": "ca-central-1a",
        "instance_ami": "ami-05b7ce93ce9fc8e32"
    },
    "eu-north-1": {
        "region": "eu-north-1",
        "zone": "eu-north-1a",
        "instance_ami": "ami-057ca637f0e971f76"
    },
    "us-east-1": {
        "region": "us-east-1",
        "zone": "us-east-1a",
        "instance_ami": "ami-09ad94ebabb1d9472"
    },
    "us-west-1": {
        "region": "us-west-1",
        "zone": "us-west-1a",
        "instance_ami": "ami-0c3a8f7521f75b65b"
    }
}


## Replace the Values of "locations" with the values outputted from the above command

In [10]:
%cat ./tf/.env.example

{
  "access_key": "<your_access_key>", 
  "secret_key": "<your_secret_key>", 
  "bootstrap_region": "ca-central-1", 
  "locations": {
    "ca-central-1": {
      "region": "ca-central-1",
      "availability_zone": "ca-central-1a",
      "instance_ami": "ami-05b7ce93ce9fc8e32"
    },
    "eu-north-1": {
      "region": "eu-north-1",
      "availability_zone": "eu-north-1a",
      "instance_ami": "ami-057ca637f0e971f76"
    },
    "us-east-1": {
      "region": "us-east-1",
      "availability_zone": "us-east-1a",
      "instance_ami": "ami-09ad94ebabb1d9472"
    },
    "us-west-1": {
      "region": "us-west-1",
      "availability_zone": "us-west-1a",
      "instance_ami": "ami-0c3a8f7521f75b65b"
    }
  },
  "app_tag": "multiregion-bacalhau-example",
  "instance_type": "g4dn.xlarge",
  "bacalhau_run_file": "bacalhau.run",
  "tailscale_key": "<your_tailscale_key>",
  "shelluser": "bacalhau-installer",
  "public_key": "~/.ssh/id_rsa.pub",
  "private_key": "~/.ssh/id_rsa"
}


## Replace The values of the keys with your own key values and also the value of instance_type if you want

In [ ]:
%%bash
cp -r .env.example .env.json

### Deploying the infra

In [ ]:
%%bash
./bulk-deploy.sh create

### Populating the Buckets with random samples from the coco-2017 val dataset
Putting 10 sample images in each bucket

In [ ]:
%%bash
python3 populate_buckets.py --sample_size 10

### Running the job

In [ ]:
%%bash
python3 job.py "person"

### Downloading the outputs

In [ ]:
%%bash
python3 download_outputs.py

### Viewing the outputs

In [1]:
import os
from PIL import Image
import ipywidgets as widgets
from IPython.display import display

# Path to the folder where the images are stored
image_folder = os.path.join(os.getcwd(), 'outputs')

# List to store the images
images = []

# Go through all the files in the folder
for filename in os.listdir(image_folder):
    # Check if the file is an image
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Open the image and append it to the list
        images.append(Image.open(os.path.join(image_folder, filename)))

# Create a slider
slider = widgets.IntSlider(min=0, max=len(images)-1, step=1)

# Function to display an image
def display_image(i):
    display(images[i])

# Link the slider to the function
widgets.interact(display_image, i=slider)

interactive(children=(IntSlider(value=0, description='i', max=9), Output()), _dom_classes=('widget-interact',)???

<function __main__.display_image(i)>